# Imports

In [1]:
# If you have not installed `wiscs` locally, run this cell
!pip install git+https://github.com/w-decker/wiscs.git --quiet # REQUIRED FOR THIS NOTEBOOK
!pip install git+https://github.com/w-decker/rinterface.git --quiet # REQUIRED FOR THIS NOTEBOOK

In [1]:
# always run this cell
import wiscs
from wiscs.simulate import DataGenerator
from wiscs.utils import make_tasks

import rinterface.rinterface as R

from src.utils import fmt_script

import numpy as np

# Generating data
Data are generated using the [`wiscs`](https://github.com/w-decker/wiscs) framework. For now, arbitrary variance parameters and design choices are set to help "build" the linear model. Once a final model has been chosen, additional power analyses can be run to determine the exact experimental design criteria.

>Please see [generate_data.ipynb](/notebooks/generate_data.ipynb) for information on how to use the `wiscs` module. 

In [2]:
task = make_tasks(low=100, high=200, n=5)
params = {'word.perceptual': 100, 'image.perceptual': 95, 'word.conceptual': 100, 'image.conceptual': 100, 'word.task': task, 'image.task': task,
        # noise parameters     
        'sd.item': None,     'sd.question': None,    'sd.subject': 20,       "sd.modality": None, "sd.error": 50, "sd.re_formula": "(1|subject)",
        # correlations among random effects    
        "corr.subject": np.array([[1]]),
        # design parameters
        'n.subject': 300, 'n.question': 5, 'n.item': 10
}
wiscs.set_params(params)

# generate data
DG = DataGenerator()
df = DG.fit_transform(seed=2025).to_pandas()

Params set successfully


/opt/homebrew/anaconda3/envs/wiscs-stats/lib/python3.12/site-packages/wiscs/simulate.py:211: UserWarning: Simulating data for MAIN hypothesis.
  warnings.warn("Simulating data for MAIN hypothesis.")


# Evaluate in R with [`rinterface`](https://github.com/w-decker/rinterface)

I have built a small interface between Python and R. Check out the repo [here](https://github.com/w-decker/rinterface). Essentially, it takes in a multiline string containing an R-valid script and runs that as a subprocess using the `Rscript` command. There's a lot more to it than that (check out the repo for examples and additional functionality), but that's the jist. For the sake of brevity, I've condensed the script we will be regularly running inside a function [`src.fmt_script()`](/notebooks/src/utils.py) (see line 144). This will keep the notebook cleaner. A few things to note on `fmt_script()`:

### What's inside `fmt_script()`?
1. Imports necessary packages, including `lme4` and `lmerTest`. 
2. Factorizes categorical variables (this is hardcoded in because we already know what variables are which)
3. Establishes treatment codes for categorical variables. You can print the script to see more details.
4. Runs the models and prints the summary of each
5. Runs `anova` on the two models

### What do _you_ need to give `fmt_script()`?
1. The R formulas for the shared and separate model as a string
2. The pandas dataframe containing the data

### What if you want to add some more code?
You can optionally specify a list of strings containing new lines of code. Each element in your list will be a new line. These are then added right before the model is run: 

```python
fmt_script(shared_f=..., separate_f=..., df=df, add=['cat("hello world")'])
```

In [4]:
script = fmt_script(shared_f="rt ~ modality + question + (1 | subject)", 
                    separate_f="rt ~ modality * question + (1 | subject)",
                    df=df)
R(script)


 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 | subject)
   Data: df

      AIC       BIC    logLik  deviance  df.resid 
 320727.0  320793.5 -160355.5  320711.0     29992 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.1634 -0.6723  0.0011  0.6751  3.7111 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept)  365.7   19.12   
 Residual             2502.4   50.02   
Number of obs: 30000, groups:  subject, 300

Fixed effects:
                Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)     324.2573     1.3113   522.3453  247.28   <2e-16 ***
modalityimage    -5.8790     0.5776 29700.0000  -10.18   <2e-16 ***
question1        24.8075     0.9133 29700.0000   27.16   <2e-16 ***
question2        51.2677     0.9133 29700.0000   56.13   <2e-16 ***
question3       -24.4599     0.9133 29700.0000  -26.78   <2e-16 ***
que

# What does the rest of this notebook look like?

Much of this notebook is evaluating different values specified in the data generation process and different formula's to specify a linear model. The point is to define a conceptually sound model that maximally aligns with the hypotheses laid out in the project. As such, the two previous codeblocks will be repeatedly run (in new cells) below. So much of what's written below is similar to you've already seen. All that's changed are values given to the data generator and the linear models.

## Starting small: 1 random effect

Here, we are modelling `subject` as a random effect. Specifically, we are giving subjects random _**intercepts**_. This allows subjects to vary in their mean reaction time inside the model. 

In [5]:
# formulas
shared_subj_re = "rt ~ modality + question + (1 | subject)"
separate_subj_re = "rt ~ modality * question + (1 | subject)"

In [6]:
script = fmt_script(shared_f=shared_subj_re, separate_f=separate_subj_re, df=df)
R(script)


 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 | subject)
   Data: df

      AIC       BIC    logLik  deviance  df.resid 
 320727.0  320793.5 -160355.5  320711.0     29992 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.1634 -0.6723  0.0011  0.6751  3.7111 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept)  365.7   19.12   
 Residual             2502.4   50.02   
Number of obs: 30000, groups:  subject, 300

Fixed effects:
                Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)     324.2573     1.3113   522.3453  247.28   <2e-16 ***
modalityimage    -5.8790     0.5776 29700.0000  -10.18   <2e-16 ***
question1        24.8075     0.9133 29700.0000   27.16   <2e-16 ***
question2        51.2677     0.9133 29700.0000   56.13   <2e-16 ***
question3       -24.4599     0.9133 29700.0000  -26.78   <2e-16 ***
que

As you can see, this model does fairly well at recovering the standard deviations we've set in the simulation.

| **Variable** | **Given** | **Estimated** |
| ------------ | --------- | ------------- |
| Subject | 20 | 19.124 |
| Residual | 50 | 50.024 |

Our shared model performed as expected relative to the separate model:

```plain
 ANOVA
Data: df
Models:
shared: rt ~ modality + question + (1 | subject)
separate: rt ~ modality * question + (1 | subject)
         npar    AIC    BIC  logLik deviance  Chisq Df Pr(>Chisq)
shared      8 320727 320793 -160355   320711                     
separate   12 320732 320832 -160354   320708 2.8824  4     0.5777
```

There were no convergence issues. The model we've specified reliably parses the variance in our data. 

# Adding another random effect

Let's add another random effect: `item`. Here, we will simulate an random _**intercept**_ for item. This means that reaction times can vary by item.

>Note that we did not have any values set in our original data setup. We will need to update the `DataGenerator` with a value for `item` denoting the standard deviation to be used as the random intercept, as well as a correlation matrix and an updated random effects formula.

In [17]:
# update the data generator
DG.fit_transform({'sd.item':20, 'sd.re_formula':'(1 | subject) + (1 | item)', 'corr.item':np.array([[1]])}, overwrite=True, seed=2025)
# convert to Pandas DataFrame
df = DG.to_pandas()

/opt/homebrew/anaconda3/envs/wiscs-stats/lib/python3.12/site-packages/wiscs/simulate.py:211: UserWarning: Simulating data for MAIN hypothesis.
  warnings.warn("Simulating data for MAIN hypothesis.")


In [18]:
shared_subj_re = "rt ~ modality + question + (1 | subject) + (1 | item)"
separate_subj_re =  "rt ~ modality * question + (1 | subject) + (1 | item)"
script = fmt_script(shared_f=shared_subj_re, separate_f=separate_subj_re, df=df)
R(script)


 SHARED model summary
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: rt ~ modality + question + (1 | subject) + (1 | item)
   Data: df

      AIC       BIC    logLik  deviance  df.resid 
 320786.2  320861.0 -160384.1  320768.2     29991 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.0850 -0.6745  0.0036  0.6738  3.7116 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept)  355.8   18.86   
 item     (Intercept)  160.6   12.67   
 Residual             2503.7   50.04   
Number of obs: 30000, groups:  subject, 300; item, 10

Fixed effects:
                Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)     319.5211     4.2123    12.0770   75.86   <2e-16 ***
modalityimage    -5.8738     0.5778 29690.9907  -10.17   <2e-16 ***
question1        26.1611     0.9135 29690.9906   28.64   <2e-16 ***
question2        50.9962     0.9135 29690.9906   55.82   <2e-16 ***
question